In [44]:
from tqdm.auto import tqdm
tokenized_dataset_path = "/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.tsv"
trectext_format="<DOC>\n<DOCNO>{}</DOCNO>\n<TEXT>\n{}\n</TEXT>\n</DOC>\n"
with open("/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.trec", 'w') as outf:    
    for line in tqdm(open(tokenized_dataset_path)):
        docno = line.split("\t")[0]
        doc_text = " ".join(line.split("\t")[1:])
        outf.write(trectext_format.format(docno,doc_text))

In [5]:
line.decode().split("\t")

['D1555982',
 'https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR',
 'The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?',
 'Science & Mathematics PhysicsThe hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?It is a good approximation to assume that the emissivity e is equal to 1 for these surfaces.  Find the radius of the star Rigel, the bright blue star in the constellation Orion that radiates energy at a rate of 2.7 x 10^32 W and has a surface temperature of 11,000 K. Assume that the star is spherical. Use Ï\x83 =... show moreFollow 3 answersAnswersRelevanceRatingNewestOldestBest Answer: Stefan-Boltzmann law states that the energy flux by radiation is proportional to the forth power of the temperature: q = Îµ Â· Ï\x83 Â· T^4 The total energy flux at a spherical surface of Radius R is Q = qÂ·Ï\x80Â·RÂ² = ÎµÂ·Ï\x83Â·T^4Â·Ï\x80Â·RÂ² Hence the radius is R = â\x88\x9a ( Q / (ÎµÂ·Ï\x83Â·T^4Â·Ï\x80) ) =

In [ ]:
from tqdm.auto import tqdm

offset_dict = {}
with open("/ssd2/arthur/TREC2019/data/docs/msmarco-docs.tsv") as inf:
    location = inf.tell()
    line = inf.readline().encode("utf-8")
    pbar = tqdm(total = 3213835)
    idx = 0 
    while line:
        did = line.decode().split("\t")[0]
        offset_dict[did] = location
        location = inf.tell()
        line = inf.readline().encode("utf-8")
        pbar.update()
        idx+=1
    pbar.close()
import pickle
pickle.dump(offset_dict, open("/ssd2/arthur/TREC2019/data/docs/msmarco-docs.tsv.offset", 'wb'))

In [5]:
from pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [58]:
#TSV to TREC 
import re
#tokenize in the same way as the documents!!
queries_file = "/ssd2/arthur/TREC2019/data/queries/test_queries.tsv"
output_file = "/ssd2/arthur/TREC2019/data/queries/test_queries.params"
param_format = "<query>\n<number>{}</number>\n<text>#combine( {} )</text>\n</query>\n"
with open(queries_file) as inf, open(output_file, 'w') as outf:
    outf.write("<parameters>\n")
    for line in inf:
        query_id = line.split("\t")[0]
        query = tokenizer.tokenize(line.split("\t")[1])
        query = ' '.join([x for x in query]).replace("##", "")
        query = re.sub(r'([^\s\w]|_)+', '', query)
        outf.write(param_format.format(query_id, query))
    outf.write("</parameters>")

In [1]:
def truncate_seq_pair(tokens_a, tokens_b, max_length=509):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


In [29]:
from tqdm.auto import tqdm
split = "train"
all_docs = []
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.top100".format(split), 'w') as outf:
    for line in tqdm(open(run_file), total = 100*len(queries)):
        topic_id, _, doc_id, _, _, _ = line.split()
        all_docs.append((topic_id, doc_id))

In [31]:
n_cpus = 36
excess_lines = len(all_docs) % n_cpus
if excess_lines > 0:
    number_of_chunks = n_cpus - 1

else:
    number_of_chunks = n_cpus
chunk_size = len(all_docs)//number_of_chunks

In [54]:
import multiprocessing as mp

pool = mp.Pool(n_cpus)
jobs = []

def process_chunk(chunk_no, n_itens, max_lines):
    starting_index = chunk_no*n_itens
    end_index = min(((chunk_no+1)*n_itens)-1, max_lines)
    outfile = "/ssd2/arthur/TREC2019/data/triples-tokenized/cut-train.top100.{}".format(chunk_no)
    with open(outfile, 'w') as outf:
        for topic_id, doc_id in tqdm(all_docs[starting_index:end_index], position = chunk_no+1):
            _id = "{}-{}".format(topic_id, doc_id)
            doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_docs))
            label = "1" if _id in qrels else "0"
            query = queries[topic_id]
            truncate_seq_pair(query, doc_content)
            tokens = ["[CLS]"] + query + ["[SEP]"] + doc_content + ["[SEP]"]
            outf.write("{}\t{}\t{}\n".format(_id, tokens, label))

def update(*a):
    pbar.update()
pbar = tqdm(total=number_of_chunks)
for i in range(number_of_chunks):
    jobs.append(pool.apply_async(process_chunk, args=(i, chunk_size, len(all_docs)), callback=update))
pbar.close()
for job in jobs:
        job.get()
pool.close()

Process ForkPoolWorker-265:
Process ForkPoolWorker-244:
Process ForkPoolWorker-262:
Process ForkPoolWorker-248:
Process ForkPoolWorker-260:
Process ForkPoolWorker-251:
Process ForkPoolWorker-257:
Process ForkPoolWorker-253:
Process ForkPoolWorker-254:
Process ForkPoolWorker-240:
Process ForkPoolWorker-238:
Process ForkPoolWorker-258:
Process ForkPoolWorker-246:
Process ForkPoolWorker-243:
Traceback (most recent call last):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/mult

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/queue

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return s

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 640, in tokenize
    tokenized_text = split_on_tokens(added_tokens, text)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-53-208ac9ba460e>", line 13, in process_chunk
    doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_docs))
Process ForkPoolWorker-278:
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 640, in tokenize
    tokenized_text = split_on_tokens(added_tokens, text)
  File "<ipython-input-53-208ac9ba460e>", line 13, in process_chunk
    doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_docs))
Process ForkPoolWorker-283:
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, 

...

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 170, in _tokenize
    for token in self.basic_tokenizer.tokenize(text, never_split=self.all_special_tokens):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 640, in tokenize
    tokenized_text = split_on_tokens(added_tokens, text)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 640, in tokenize
    toke

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in <genexpr>
    else [token] for token in tokenized_text), [])
KeyboardInterrupt
Process ForkPoolWorker-252:
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 170, in _tokenize
    for token in self.basic_tokenizer.tokenize(text, never_split=self.all_special_tokens):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in <genexpr>
    else [token] for token in tokenized_text), [])
  File "/home/arthur/min

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in split_on_tokens
    else [token] for token in tokenized_text), [])
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 273, in tokenize
    split_tokens.extend(self._run_split_on_punc(token))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 258, in tokenize
    text = self._clean_text(text)
KeyboardInterrupt
  File "<ipython-input-53-208ac9ba460e>", line 13, in process_chunk
    doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_d

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 273, in tokenize
    split_tokens.extend(self._run_split_on_punc(token))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in <genexpr>
    else [token] for token in tokenized_text), [])
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 272, in tokenize
    token = self._run_strip_accents(token)
  File "<ipython-input-53-208ac9ba460e>", line 13, in process_chunk
    doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_docs))
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in <genexpr>
    else [token] for token in tokenized_text), [])
KeyboardInterrupt
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/t

  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 637, in <genexpr>
    else [token] for token in tokenized_text), [])
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 170, in _tokenize
    for token in self.basic_tokenizer.tokenize(text, never_split=self.all_special_tokens):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 299, in _run_split_on_punc
    if _is_punctuation(char):
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_utils.py", line 640, in tokenize
    tokenized_text = split_on_tokens(added_tokens, text)
  File "/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/pytorch_transformers/tokenization_bert.py", line 170, in _tokenize
    for token in self.basic_tokenizer.tokenize(text, never_split=self.all_sp

In [56]:
query

['watts', '&', 'browning', 'engineers']

In [50]:
#Generate top100 triples from runs
from collections import defaultdict, Counter

split = "train"

# tokenized_queries = {}
# run_file = "/ssd2/arthur/TREC2019/data/runs/indri_{}.run".format(split)
# qrels_file = "/ssd2/arthur/TREC2019/data/qrels/{}_qrels".format(split)
# qrels = set()
# for line in tqdm(open(qrels_file)):
#     query_id, _, doc_id, label = line.strip().split()
#     tuple_id = "{}-{}".format(query_id, doc_id)
#     if label=="1":
#         qrels.add(tuple_id)

# queries = {}
# for line in tqdm(open("/ssd2/arthur/TREC2019/data/queries/{}_queries.tsv".format(split))):
#     query_id, query_text = line.strip().split("\t")
#     queries[query_id] = tokenizer.tokenize(query_text)

tokenized_docs = "/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.tsv"
processed_per_topic = Counter()
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.top100".format(split), 'w') as outf:
    for line in tqdm(open(run_file), total = 100*len(queries)):
        topic_id, _, doc_id, _, _, _ = line.split()
        tuple_id = "{}-{}".format(topic_id, doc_id)
        doc_content = tokenizer.tokenize(getcontent(doc_id, tokenized_docs))
        label = "1" if tuple_id in qrels else "0"
        query = queries[topic_id]
        truncate_seq_pair(query, doc_content)
        tokens = ["[CLS]"] + query + ["[SEP]"] + doc_content + ["[SEP]"]
        outf.write("{}\t{}\t{}\n".format(tuple_id, tokens, label))

KeyboardInterrupt: 

In [57]:
#top 10\
from collections import defaultdict
import random
from tqdm.auto import tqdm
from pytorch_transformers import BertTokenizer
import pickle
offset_dict = pickle.load(open("/ssd2/arthur/TREC2019/data/docs/tokenized-msmarco-docs.tsv.offset", 'rb'))

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
split = "train"
qrels_file = "/ssd2/arthur/TREC2019/data/qrels/{}_qrels".format(split)

qrels = defaultdict(lambda:set())
for line in open(qrels_file):
    query_id, _, doc_id, label = line.strip().split()
    tuple_id = "{}-{}".format(query_id, doc_id)
    if label=="1":
        qrels[query_id].add(doc_id)

last_topic = -1
top_100 = []
n_negatives = 10
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.{}neg".format(split, n_negatives), 'w') as outf:
    for line in tqdm(open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.top100".format(split)), total = 100*len(qrels)):
        guid, doc, label = line.strip().split("\t")
        topic_id, doc_id = guid.split("-")
        if topic_id != last_topic and last_topic != -1:
            for positive in qrels[last_topic]:
                tokenized_positive = tokenizer.tokenize(getcontent(positive, tokenized_docs))
                outf.write("{}-{}\t{}\t1\n".format(last_topic, positive, tokenized_positive))
                random_neg = random.sample(top_100, n_negatives)
                for neg in random_neg:
                    outf.write("{}\t{}\t0\n".format(neg[0], neg[1]))
            top_100 = []
        if label != "1":
            top_100.append((guid, eval(doc), label))
        last_topic = topic_id

In [49]:
def truncate_seq_pair(tokens_a, tokens_b, max_length=509):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

            

In [6]:
from tqdm.auto import tqdm
from pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
split = "train"
queries = {}
for line in tqdm(open("/ssd2/arthur/TREC2019/data/queries/{}_queries.tsv".format(split))):
    query_id, query_text = line.strip().split("\t")
    queries[query_id] = tokenizer.tokenize(query_text)


In [9]:
def truncate_seq_pair(tokens_a, tokens_b, max_length=509):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [10]:
with open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.10neg.fix".format(split), 'w') as outf:
    for line in open("/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.10neg".format(split)):
        if "[SEP]" in line:
            outf.write(line)
        else:
            _id, doc, label = line.strip().split("\t")
            topic_id, doc_id = _id.split("-")
            query_text = queries[topic_id]
            doc = eval(doc)
            truncate_seq_pair(query_text, doc)
            final_doc = ["[CLS]"] + query_text + ["[SEP]"] + doc + ["[SEP]"]
            outf.write(f"{_id}\t{final_doc}\t{label}\n")